# Disciplina: Introdução à Inteligência Artificial
Instituto Federal do Norte de Minas Gerais Campus Montes Claros <br/>
Curso: Ciência da Computação <br/>
Aluno: David Walter Jansen <br/>

**Experimento 1: um algoritmo genético simples, com cruzamento**
<ol>
<li> Geração da população inicial, de tamanho N, de forma aleatória, entre -5.12 e 5.12. Sugestão N=20 </li>
<li> Seleção por torneio 2 a 2. Lembrando que os indivíduos selecionados podem ser repetidos. </li>
<li> Realizar N/2 cruzamentos com 90% de chance para os indivíduos campeões do torneio. Cada cruzamento produz 2 filhos. E cada filho tem um percentual alpha de participação de um pai, e 1-a lpha de participação do pai 2. </li>
<li> Realizar a mutação, sendo que, a taxa de mutação seja de 10% (por exemplo, 10 a cada 100 indivíduos devem sofrer mutação).</li>
- Da população selecionada no item 3, você deve escolher 10% deles e realizar uma perturbação de + ou - 5% do valor do indivíduo. Por exemplo, selecione um indivíduo, ele deve contar x_1 e x_2: <br/>
-> Selecione qual variável/dimensão você deseja alterar <br/>
-> Selecione qual operação você irá realizar +5% ou -5% <br/>

<li> Entre os pais e filhos, escolha os novos N indivíduos, que irão para a próxima geração</li>
<li> Exiba a cada geração o melhor indivíduo (x e f(x))</li>
<li> Repita 2 a 6 até atingir o número máximo de gerações. Sugestão: 100 gerações.</li>
</ol>
Depois do algoritmo pronto, execute-o 30 vezes. Para cada execução salve o melhor indivíduo ao final das gerações. Portanto, serão 30 indivíduos diferentes

In [94]:
import numpy as np
import random
import math

In [95]:
num_ger = 100 #numero máximo de gerações que AG executará
num_pop = 20  #haverá 20 individuos na populacao
chancem = 0.1 #10% dos individuos selecionados sofrerão mutação
chancec = 0.9 #90% dos individuos selecionados sofrerão cruzamento
num_dim=2
xmin=-5.12
xmax=5.12

In [96]:
#define a função para gerar a pop inicial de forma aleatória
def gera_pop_inicial(num_pop, n_dim,xmin, xmax):
    return np.random.uniform(xmin,xmax,(num_pop,num_dim))

In [97]:
def inverte(fx):
    hx = min(fx);
    desl = 10**-0.2 - hx;
    return 1/( fx + desl);

def rastrigin(X, **kwargs):
    
    A = kwargs.get('A')
    [row, col] = X.shape
    fx= np.zeros(row)
    for i in range(row):
        for j in range(col):
            fx[i] += X[i,j]**2 - A * np.cos(2 * math.pi * X[i,j])
    #print(X)
    return A*col + fx

In [98]:
def popTorneio(ind, X):
    [row, col] = X.shape
    Xt = np.zeros((row,col))
    
    for i in range(row):
        Xt[i] = X[ind[i].astype(int)]
        
    return Xt

In [99]:
def torneio(fx):
    ind = np.zeros(len(fx))
    for i in range(len(fx)):
        #sorteia dois individuos e seleciona o melhor com relação ao fitness, armazena o indice do vendedor
        i1 = random.randint(0,len(fx)-1)
        i2 = random.randint(0,len(fx)-1)
        #print(i1,i2)
        if fx[i1] > fx[i2]:
            ind[i] = i1
        else:
            ind[i] = i2
    return ind

In [100]:
def mutacao(X, chance):
    [row, col] = X.shape
    Xn = np.zeros((row,col))
    for i in range(row):
        #sorteia um individuo e verifica se ele deve ser mutado
        i1 = random.randint(0,row-1)
        
        if random.random() <= chance:
            #print(i1)
            if random.random()>0.5:
                Xn[i,:] = X[i,:] + X[i,:]*0.05
            else:
                Xn[i,:] = X[i,:] - X[i,:]*0.05
        else:
            Xn[i,:] = X[i,:]
    return Xn

In [101]:
def cruzamento1(X, chance):
    r = X.shape[0]
    c = X.shape[1]
    Xfilho = np.zeros((r,c))
    filho = 0
    alpha = 0
    while alpha == 0:
        alpha = random.random()
        
    while(filho < 20):
        if random.random() <= chance:
            i1 = random.randint(0,r-1)
            i2 = random.randint(0,r-1)
            Xfilho[filho,:] = alpha*X[i1,:] + (1-alpha)*X[i2,:]
            filho = filho + 1
            Xfilho[filho,:] = alpha*X[i2,:] + (1-alpha)*X[i1,:]
            filho = filho + 1

    return Xfilho

def cruzamento2(X, chance):
    r = X.shape[0]
    c = X.shape[1]
    Xfilho = np.zeros((r,c))
    filho = 0
    alpha = 0.9
    for i in range(int(r/2)):
        i1 = random.randint(0,r-1)
        i2 = random.randint(0,r-1)
        Xfilho[filho,:] = alpha*X[i1,:] + (1-alpha)*X[i2,:]
        filho = filho + 1
        Xfilho[filho,:] = alpha*X[i2,:] + (1-alpha)*X[i1,:]
        filho = filho + 1
    
    return Xfilho

def cruzamento3(X, chance):
    r = X.shape[0]
    c = X.shape[1]
    Xfilho = np.zeros((r,c))
    filho = 0
    for i in range(int(r/2)):
        alpha = random.random()
        i1 = random.randint(0,r-1)
        i2 = random.randint(0,r-1)
        Xfilho[filho,:] = alpha*X[i1,:] + (1-alpha)*X[i2,:]
        filho = filho + 1
        Xfilho[filho,:] = alpha*X[i2,:] + (1-alpha)*X[i1,:]
        filho = filho + 1
    
    return Xfilho

In [102]:
def update(X, z, Xn, zn):
    auxIndX = np.argmin(z) #melhor z
    auxIndXn = np.argmax(zn) #pior z

    #substitui o pior da geração atual, pelos melhor da geração anterior
    Xn[auxIndXn, :] = X[auxIndX, :]
    
    return Xn

In [103]:
def algoritimoGenetico(cruzamento):
    #armazena o melhor elemento de cada geração
    melhoresIndividuos = np.zeros((30,2))
    melhoresFx = np.zeros((30,2))

    for exect in range(30):
        print("\n----------%dº Execução----------\n" %(exect + 1))
        #Passo 1: geração da população inicial
        X = gera_pop_inicial(num_pop,num_dim, xmin, xmax)
        
        #agora devemos avaliar todos os individuos da população inicial

        #Passo 7: Repetição
        for i in range(num_ger):
            #descomentar as duas linhas abaixo para mostrar o melhor a cada geração]
            z = rastrigin(X, A=10)
            fx = inverte(z)
            melhorind =  np.argmin(z)
            melhorfx = fx[melhorind]

            print("%dº geração: " %(i+1))
            print("Melhor individuo:", end=' ')
            print(X[melhorind])
            print("Z para esse individuo:", end=' ')
            print(z[melhorind])
            print("Fx para esse individuo:", end=' ')
            print(melhorfx, end='\n')

            #Passo 2: realização do torneio
            ind = torneio(fx)
            Xt = popTorneio(ind, X)

            #Passo 3:Cruzamento após o torneio
            if cruzamento == 1 :
                Xf =  cruzamento1(Xt, chancec)
            elif cruzamento == 2 :
                Xf =  cruzamento2(Xt, chancec)
            elif cruzamento == 3 :
                Xf =  cruzamento3(Xt, chancec)

            #Passo 4: Mutação
            Xn = mutacao(Xf, 0.1) 

            zn = rastrigin(Xn, A=10)
            fxn = inverte(zn)

            #Passo 5: Atualização
            X = update(X, z, Xn, zn)

        #armazena o melhor da g-ésima execução após 100 gerações. Nesse caso, será da função, ao inves da inversão, 
        #pois a inversão depende do mínimo este pode variar a cada iteração 
        z = rastrigin(X, A=10)
        fx = inverte(z)
        melhorind =  np.argmin(z)
        melhoresIndividuos[exect] = X[melhorind]
        melhorfx = fx[melhorind]
        melhoresFx[exect] = melhorfx
        
    return melhoresIndividuos, melhoresFx

In [104]:
def convergeTest(value1, value2, limit):
    if (abs(value1) + abs(value2)) < limit :
        return True
    return False

<h1>Execução do Algoritmo</h1>

<p>Para esse trabalho serão realizados testes para 3 funções de cruzamento:</p>
<ol>
    <li>Atribuição de chance para geração de filhos após o sorteio dos possíveis pais. A participação é previamente definida para cada pai.</li>
    <li>O sorteio dos pais irá produzir dois filhos. A participação é previamente definida para cada pai.</li>
    <li>A participação de cada pai nos filhos é definida por sorteio.</li>
</ol>

<p>Cada teste, executa 30 vezes o algoritmo genético utilizando uma das funções de cruzamento. Cada execução exibe o melhor indivíduo de cada geração e seu melhor fitness, além de retornar o melhor indivíduo após 100 gerações.</p>

<h2> Teste: cruzamento1 </h2>

<h3>Melhores de cada geração</h3>

In [105]:
bestsIwithC1, bestsFxwithC1 = algoritimoGenetico(1)


----------1º Execução----------

1º geração: 
Melhor individuo: [ 0.12010711 -1.74271006]
Z para esse individuo: 16.22426638887585
Fx para esse individuo: 1.5848931924611112
2º geração: 
Melhor individuo: [ 1.9357501 -1.1452855]
Z para esse individuo: 9.747758705342772
Fx para esse individuo: 1.5848931924611112
3º geração: 
Melhor individuo: [ 2.02232918 -0.98262324]
Z para esse individuo: 5.213164302569476
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [ 2.02232918 -0.98262324]
Z para esse individuo: 5.213164302569476
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [ 2.02232918 -0.98262324]
Z para esse individuo: 5.213164302569476
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [ 2.01012529 -0.99438981]
Z para esse individuo: 5.055856987034911
Fx para esse individuo: 1.5848931924611134
7º geração: 
Melhor individuo: [ 1.99543814 -1.00206098]
Z para esse individuo: 4.990845589091943
Fx para esse individuo: 

Z para esse individuo: 0.07624244695018945
Fx para esse individuo: 1.5848931924611134
93º geração: 
Melhor individuo: [-0.00192924 -0.01859765]
Z para esse individuo: 0.06927916851804383
Fx para esse individuo: 1.5848931924611134
94º geração: 
Melhor individuo: [-0.00192708 -0.01857678]
Z para esse individuo: 0.06912394575596892
Fx para esse individuo: 1.5848931924611134
95º geração: 
Melhor individuo: [-0.00192438 -0.01855075]
Z para esse individuo: 0.06893057943591074
Fx para esse individuo: 1.5848931924611134
96º geração: 
Melhor individuo: [-0.00192438 -0.01855075]
Z para esse individuo: 0.06893057943591074
Fx para esse individuo: 1.5848931924611134
97º geração: 
Melhor individuo: [-0.00188182 -0.01814045]
Z para esse individuo: 0.06591834233831051
Fx para esse individuo: 1.5848931924611134
98º geração: 
Melhor individuo: [-0.00188182 -0.01814045]
Z para esse individuo: 0.06591834233831051
Fx para esse individuo: 1.5848931924611134
99º geração: 
Melhor individuo: [-0.00182941 -0.01

Melhor individuo: [0.00804819 0.99543588]
Z para esse individuo: 1.0078520155348798
Fx para esse individuo: 1.5848931924611134
49º geração: 
Melhor individuo: [0.00804819 0.99543588]
Z para esse individuo: 1.0078520155348798
Fx para esse individuo: 1.5848931924611134
50º geração: 
Melhor individuo: [0.00804819 0.99543588]
Z para esse individuo: 1.0078520155348798
Fx para esse individuo: 1.5848931924611134
51º geração: 
Melhor individuo: [0.00804819 0.99543588]
Z para esse individuo: 1.0078520155348798
Fx para esse individuo: 1.5848931924611134
52º geração: 
Melhor individuo: [0.00804819 0.99543588]
Z para esse individuo: 1.0078520155348798
Fx para esse individuo: 1.5848931924611134
53º geração: 
Melhor individuo: [0.00804819 0.99543588]
Z para esse individuo: 1.0078520155348798
Fx para esse individuo: 1.5848931924611134
54º geração: 
Melhor individuo: [0.00804819 0.99543588]
Z para esse individuo: 1.0078520155348798
Fx para esse individuo: 1.5848931924611134
55º geração: 
Melhor indivi

35º geração: 
Melhor individuo: [ 0.00551965 -0.99474612]
Z para esse individuo: 1.001011725858156
Fx para esse individuo: 1.5848931924611134
36º geração: 
Melhor individuo: [ 0.00551965 -0.99474612]
Z para esse individuo: 1.001011725858156
Fx para esse individuo: 1.5848931924611134
37º geração: 
Melhor individuo: [ 0.00551965 -0.99474612]
Z para esse individuo: 1.001011725858156
Fx para esse individuo: 1.5848931924611134
38º geração: 
Melhor individuo: [ 0.00551965 -0.99474612]
Z para esse individuo: 1.001011725858156
Fx para esse individuo: 1.5848931924611134
39º geração: 
Melhor individuo: [ 0.00551965 -0.99474612]
Z para esse individuo: 1.001011725858156
Fx para esse individuo: 1.5848931924611134
40º geração: 
Melhor individuo: [ 0.00551965 -0.99474612]
Z para esse individuo: 1.001011725858156
Fx para esse individuo: 1.5848931924611134
41º geração: 
Melhor individuo: [ 0.00551965 -0.99474612]
Z para esse individuo: 1.001011725858156
Fx para esse individuo: 1.5848931924611134
42º ge

Fx para esse individuo: 1.5848931924611134
92º geração: 
Melhor individuo: [-0.05135824 -0.99049967]
Z para esse individuo: 1.5176904774538151
Fx para esse individuo: 1.5848931924611134
93º geração: 
Melhor individuo: [-0.05135824 -0.99049967]
Z para esse individuo: 1.5176904774538151
Fx para esse individuo: 1.5848931924611134
94º geração: 
Melhor individuo: [-0.05135824 -0.99049967]
Z para esse individuo: 1.5176904774538151
Fx para esse individuo: 1.5848931924611134
95º geração: 
Melhor individuo: [-0.05135824 -0.99049967]
Z para esse individuo: 1.5176904774538151
Fx para esse individuo: 1.5848931924611134
96º geração: 
Melhor individuo: [-0.05135824 -0.99049967]
Z para esse individuo: 1.5176904774538151
Fx para esse individuo: 1.5848931924611134
97º geração: 
Melhor individuo: [-0.05135824 -0.99049967]
Z para esse individuo: 1.5176904774538151
Fx para esse individuo: 1.5848931924611134
98º geração: 
Melhor individuo: [-0.05135824 -0.99049967]
Z para esse individuo: 1.5176904774538151

...

35º geração: 
Melhor individuo: [ 0.98604651 -1.00958247]
Z para esse individuo: 2.0480718595375187
Fx para esse individuo: 1.5848931924611134
36º geração: 
Melhor individuo: [ 0.98604651 -1.00958247]
Z para esse individuo: 2.0480718595375187
Fx para esse individuo: 1.5848931924611134
37º geração: 
Melhor individuo: [ 0.98604651 -1.00958247]
Z para esse individuo: 2.0480718595375187
Fx para esse individuo: 1.5848931924611134
38º geração: 
Melhor individuo: [ 0.98604651 -1.00958247]
Z para esse individuo: 2.0480718595375187
Fx para esse individuo: 1.5848931924611134
39º geração: 
Melhor individuo: [ 0.98604651 -1.00958247]
Z para esse individuo: 2.0480718595375187
Fx para esse individuo: 1.5848931924611134
40º geração: 
Melhor individuo: [ 0.98604651 -1.00958247]
Z para esse individuo: 2.0480718595375187
Fx para esse individuo: 1.5848931924611134
41º geração: 
Melhor individuo: [ 0.98604651 -1.00958247]
Z para esse individuo: 2.0480718595375187
Fx para esse individuo: 1.5848931924611134

16º geração: 
Melhor individuo: [-0.00910197  0.09357106]
Z para esse individuo: 1.7042497231272797
Fx para esse individuo: 1.5848931924611134
17º geração: 
Melhor individuo: [-0.0094223   0.09106671]
Z para esse individuo: 1.618724255976229
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [-0.0094223   0.09106671]
Z para esse individuo: 1.618724255976229
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [-0.0094223   0.09106671]
Z para esse individuo: 1.618724255976229
Fx para esse individuo: 1.5848931924611134
20º geração: 
Melhor individuo: [-0.00871006  0.08888374]
Z para esse individuo: 1.5422952712023559
Fx para esse individuo: 1.5848931924611134
21º geração: 
Melhor individuo: [-0.00888922  0.08706226]
Z para esse individuo: 1.4825120933447735
Fx para esse individuo: 1.5848931924611134
22º geração: 
Melhor individuo: [-0.00857994  0.08493517]
Z para esse individuo: 1.4123222052187145
Fx para esse individuo: 1.5848931924611134
23

Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [ 2.05506125 -0.06111662]
Z para esse individuo: 5.547799779696254
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [ 2.04868909 -0.0585015 ]
Z para esse individuo: 5.332841974905332
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [ 2.04868909 -0.0585015 ]
Z para esse individuo: 5.332841974905332
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [ 2.04868909 -0.0585015 ]
Z para esse individuo: 5.332841974905332
Fx para esse individuo: 1.5848931924611134
7º geração: 
Melhor individuo: [ 2.04868909 -0.0585015 ]
Z para esse individuo: 5.332841974905332
Fx para esse individuo: 1.5848931924611134
8º geração: 
Melhor individuo: [ 1.94657066 -0.05615955]
Z para esse individuo: 4.966634248131939
Fx para esse individuo: 1.5848931924611134
9º geração: 
Melhor individuo: [ 1.98117671 -0.05754683]
Z para esse individuo: 4.644830597089847
Fx para esse 

Fx para esse individuo: 1.5848931924611134
17º geração: 
Melhor individuo: [ 0.00088269 -0.00235742]
Z para esse individuo: 0.0012571095170983426
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [ 0.00088269 -0.00235742]
Z para esse individuo: 0.0012571095170983426
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [ 0.00086452 -0.00229105]
Z para esse individuo: 0.0011895975916331736
Fx para esse individuo: 1.5848931924611134
20º geração: 
Melhor individuo: [ 0.00086452 -0.00229105]
Z para esse individuo: 0.0011895975916331736
Fx para esse individuo: 1.5848931924611134
21º geração: 
Melhor individuo: [ 0.00077465 -0.002287  ]
Z para esse individuo: 0.0011566975176151573
Fx para esse individuo: 1.5848931924611134
22º geração: 
Melhor individuo: [ 0.00081504 -0.00218655]
Z para esse individuo: 0.0010802876463813504
Fx para esse individuo: 1.5848931924611134
23º geração: 
Melhor individuo: [ 0.00078573 -0.00212877]
Z para esse individuo: 

Z para esse individuo: 2.040966710208501
Fx para esse individuo: 1.5848931924611134
31º geração: 
Melhor individuo: [-0.99459561  1.01099773]
Z para esse individuo: 2.040966710208501
Fx para esse individuo: 1.5848931924611134
32º geração: 
Melhor individuo: [-0.99459561  1.01099773]
Z para esse individuo: 2.040966710208501
Fx para esse individuo: 1.5848931924611134
33º geração: 
Melhor individuo: [-0.99459561  1.01099773]
Z para esse individuo: 2.040966710208501
Fx para esse individuo: 1.5848931924611134
34º geração: 
Melhor individuo: [-0.99459561  1.01099773]
Z para esse individuo: 2.040966710208501
Fx para esse individuo: 1.5848931924611134
35º geração: 
Melhor individuo: [-0.99459561  1.01099773]
Z para esse individuo: 2.040966710208501
Fx para esse individuo: 1.5848931924611134
36º geração: 
Melhor individuo: [-0.99459561  1.01099773]
Z para esse individuo: 2.040966710208501
Fx para esse individuo: 1.5848931924611134
37º geração: 
Melhor individuo: [-0.99459561  1.01099773]
Z para

<h3>Melhores Indivíduos de cada Execução</h3>

In [106]:
bestsIwithC1

array([[ 1.99108697e+00, -1.00238875e+00],
       [-1.78011840e-03, -1.71600969e-02],
       [ 8.57532677e-03, -1.85161754e-03],
       [ 8.04818831e-03,  9.95435885e-01],
       [ 1.00207032e+00, -9.87733487e-01],
       [ 5.51964830e-03, -9.94746120e-01],
       [-5.13582421e-02, -9.90499668e-01],
       [ 9.93563740e-01,  9.96416491e-01],
       [ 9.43871061e-01, -1.04067307e+00],
       [ 2.09286913e-02,  9.94526618e-01],
       [-3.89718374e-03,  9.94964978e-01],
       [-1.47414684e-03, -9.95094428e-01],
       [ 9.92512250e-01, -4.96735294e-02],
       [ 9.30808353e-01, -1.05052929e+00],
       [ 5.76970823e-02,  9.91765880e-01],
       [-1.88625153e-02,  7.18373870e-03],
       [-1.04996012e+00, -9.31840653e-01],
       [ 1.63588040e-02,  6.17880600e-03],
       [-8.79189409e-03,  9.95535858e-01],
       [-6.95480280e-02, -9.89930489e-01],
       [ 9.83278757e-01, -1.00674865e+00],
       [ 2.39440869e-02, -1.12297481e-03],
       [-3.36535216e-03,  3.31544142e-02],
       [ 9.

<h3>Melhores Fitness de cada Execução</h3>


In [107]:
bestsFxwithC1

array([[1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319]])

<h2> Teste: cruzamento2 </h2>

<h3>Melhores de cada geração</h3>

In [108]:
bestsIwithC2, bestsFxwithC2 = algoritimoGenetico(2)


----------1º Execução----------

1º geração: 
Melhor individuo: [-0.9868469   3.03172031]
Z para esse individuo: 10.397279995800975
Fx para esse individuo: 1.5848931924611112
2º geração: 
Melhor individuo: [-0.9868469   3.03172031]
Z para esse individuo: 10.397279995800975
Fx para esse individuo: 1.5848931924611112
3º geração: 
Melhor individuo: [-1.02162999  3.02083858]
Z para esse individuo: 10.346997150491081
Fx para esse individuo: 1.5848931924611112
4º geração: 
Melhor individuo: [-1.00854829  3.01811116]
Z para esse individuo: 10.20526277535653
Fx para esse individuo: 1.5848931924611112
5º geração: 
Melhor individuo: [-0.98813172  3.01222243]
Z para esse individuo: 10.107152790954556
Fx para esse individuo: 1.5848931924611158
6º geração: 
Melhor individuo: [-0.98813172  3.01222243]
Z para esse individuo: 10.107152790954556
Fx para esse individuo: 1.5848931924611158
7º geração: 
Melhor individuo: [-0.98813172  3.01222243]
Z para esse individuo: 10.107152790954556
Fx para esse ind

Melhor individuo: [-1.01216807  0.97743738]
Z para esse individuo: 2.109398599507891
Fx para esse individuo: 1.5848931924611134
85º geração: 
Melhor individuo: [-1.01216807  0.97743738]
Z para esse individuo: 2.1093985995078874
Fx para esse individuo: 1.5848931924611134
86º geração: 
Melhor individuo: [-1.01216807  0.97743738]
Z para esse individuo: 2.1093985995078874
Fx para esse individuo: 1.5848931924611134
87º geração: 
Melhor individuo: [-1.01216807  0.97743738]
Z para esse individuo: 2.109398599507884
Fx para esse individuo: 1.5848931924611134
88º geração: 
Melhor individuo: [-1.01216807  0.97743738]
Z para esse individuo: 2.109398599507884
Fx para esse individuo: 1.5848931924611134
89º geração: 
Melhor individuo: [-1.01216807  0.97743738]
Z para esse individuo: 2.109398599507884
Fx para esse individuo: 1.5848931924611134
90º geração: 
Melhor individuo: [-1.01216807  0.97743738]
Z para esse individuo: 2.109398599507884
Fx para esse individuo: 1.5848931924611134
91º geração: 
Melh

85º geração: 
Melhor individuo: [-0.99443691 -0.99554538]
Z para esse individuo: 1.9900403546985466
Fx para esse individuo: 1.5848931924611134
86º geração: 
Melhor individuo: [-0.99443691 -0.99554538]
Z para esse individuo: 1.9900403546985466
Fx para esse individuo: 1.5848931924611134
87º geração: 
Melhor individuo: [-0.99443691 -0.99554538]
Z para esse individuo: 1.9900403546985466
Fx para esse individuo: 1.5848931924611134
88º geração: 
Melhor individuo: [-0.99443691 -0.99554538]
Z para esse individuo: 1.9900403546985466
Fx para esse individuo: 1.5848931924611134
89º geração: 
Melhor individuo: [-0.99443691 -0.99554538]
Z para esse individuo: 1.9900403546985466
Fx para esse individuo: 1.5848931924611134
90º geração: 
Melhor individuo: [-0.99443691 -0.99554538]
Z para esse individuo: 1.9900403546985466
Fx para esse individuo: 1.5848931924611134
91º geração: 
Melhor individuo: [-0.99443691 -0.99554538]
Z para esse individuo: 1.9900403546985466
Fx para esse individuo: 1.5848931924611134

43º geração: 
Melhor individuo: [ 0.95450898 -2.00614896]
Z para esse individuo: 5.348899570710124
Fx para esse individuo: 1.5848931924611134
44º geração: 
Melhor individuo: [ 0.95489215 -2.00695429]
Z para esse individuo: 5.34818518428736
Fx para esse individuo: 1.5848931924611134
45º geração: 
Melhor individuo: [ 0.95523717 -2.00767943]
Z para esse individuo: 5.347809764477621
Fx para esse individuo: 1.5848931924611134
46º geração: 
Melhor individuo: [ 0.95523717 -2.00767943]
Z para esse individuo: 5.347809764477621
Fx para esse individuo: 1.5848931924611134
47º geração: 
Melhor individuo: [ 0.95523717 -2.00767943]
Z para esse individuo: 5.347809764477621
Fx para esse individuo: 1.5848931924611134
48º geração: 
Melhor individuo: [ 0.95523717 -2.00767943]
Z para esse individuo: 5.347809764477621
Fx para esse individuo: 1.5848931924611134
49º geração: 
Melhor individuo: [ 0.95523717 -2.00767943]
Z para esse individuo: 5.347809764477621
Fx para esse individuo: 1.5848931924611134
50º ger

Z para esse individuo: 2.4287601145500517
Fx para esse individuo: 1.5848931924611134
55º geração: 
Melhor individuo: [-1.01105507  0.95054681]
Z para esse individuo: 2.4287601145500517
Fx para esse individuo: 1.5848931924611134
56º geração: 
Melhor individuo: [-1.01105507  0.95054681]
Z para esse individuo: 2.4287601145500517
Fx para esse individuo: 1.5848931924611134
57º geração: 
Melhor individuo: [-1.01105507  0.95054681]
Z para esse individuo: 2.4287601145500517
Fx para esse individuo: 1.5848931924611134
58º geração: 
Melhor individuo: [-1.01105507  0.95054681]
Z para esse individuo: 2.4287601145500517
Fx para esse individuo: 1.5848931924611134
59º geração: 
Melhor individuo: [-1.01105507  0.95054681]
Z para esse individuo: 2.4287601145500517
Fx para esse individuo: 1.5848931924611134
60º geração: 
Melhor individuo: [-1.01105507  0.95054681]
Z para esse individuo: 2.4287601145500517
Fx para esse individuo: 1.5848931924611134
61º geração: 
Melhor individuo: [-1.01105507  0.95054681]

...

Z para esse individuo: 5.008039046092772
Fx para esse individuo: 1.5848931924611134
67º geração: 
Melhor individuo: [-0.9833009   1.99556596]
Z para esse individuo: 5.008039046092772
Fx para esse individuo: 1.5848931924611134
68º geração: 
Melhor individuo: [-0.9833009   1.99556596]
Z para esse individuo: 5.008039046092772
Fx para esse individuo: 1.5848931924611134
69º geração: 
Melhor individuo: [-0.9833009   1.99556596]
Z para esse individuo: 5.008039046092772
Fx para esse individuo: 1.5848931924611134
70º geração: 
Melhor individuo: [-0.9833009   1.99556596]
Z para esse individuo: 5.008039046092772
Fx para esse individuo: 1.5848931924611134
71º geração: 
Melhor individuo: [-0.9833009   1.99556596]
Z para esse individuo: 5.008039046092772
Fx para esse individuo: 1.5848931924611134
72º geração: 
Melhor individuo: [-0.9833009   1.99556596]
Z para esse individuo: 5.008039046092772
Fx para esse individuo: 1.5848931924611134
73º geração: 
Melhor individuo: [-0.9833009   1.99556596]
Z para

Z para esse individuo: 2.113412866116221
Fx para esse individuo: 1.5848931924611134
81º geração: 
Melhor individuo: [-0.98949564  0.07558997]
Z para esse individuo: 2.113412866116221
Fx para esse individuo: 1.5848931924611134
82º geração: 
Melhor individuo: [-0.98949564  0.07558997]
Z para esse individuo: 2.113412866116221
Fx para esse individuo: 1.5848931924611134
83º geração: 
Melhor individuo: [-0.98949564  0.07558997]
Z para esse individuo: 2.113412866116221
Fx para esse individuo: 1.5848931924611134
84º geração: 
Melhor individuo: [-0.98949564  0.07558997]
Z para esse individuo: 2.113412866116221
Fx para esse individuo: 1.5848931924611134
85º geração: 
Melhor individuo: [-0.98949564  0.07558997]
Z para esse individuo: 2.113412866116221
Fx para esse individuo: 1.5848931924611134
86º geração: 
Melhor individuo: [-0.98949564  0.07558997]
Z para esse individuo: 2.113412866116221
Fx para esse individuo: 1.5848931924611134
87º geração: 
Melhor individuo: [-0.98949564  0.07558997]
Z para

Z para esse individuo: 5.006491141311638
Fx para esse individuo: 1.5848931924611134
95º geração: 
Melhor individuo: [0.98250303 1.99213462]
Z para esse individuo: 5.006491141311638
Fx para esse individuo: 1.5848931924611134
96º geração: 
Melhor individuo: [0.98250303 1.99213462]
Z para esse individuo: 5.006491141311638
Fx para esse individuo: 1.5848931924611134
97º geração: 
Melhor individuo: [0.98250303 1.99213462]
Z para esse individuo: 5.006491141311638
Fx para esse individuo: 1.5848931924611134
98º geração: 
Melhor individuo: [0.98250303 1.99213462]
Z para esse individuo: 5.006491141311638
Fx para esse individuo: 1.5848931924611134
99º geração: 
Melhor individuo: [0.98250303 1.99213462]
Z para esse individuo: 5.006491141311638
Fx para esse individuo: 1.5848931924611134
100º geração: 
Melhor individuo: [0.98250303 1.99213462]
Z para esse individuo: 5.006491141311638
Fx para esse individuo: 1.5848931924611134

----------25º Execução----------

1º geração: 
Melhor individuo: [-2.80805

Melhor individuo: [-0.04238348  0.93805071]
Z para esse individuo: 1.9822505202521157
Fx para esse individuo: 1.5848931924611134
9º geração: 
Melhor individuo: [-0.04238348  0.93805071]
Z para esse individuo: 1.9822505202521157
Fx para esse individuo: 1.5848931924611134
10º geração: 
Melhor individuo: [-0.04187894  0.9804154 ]
Z para esse individuo: 1.3827860796116198
Fx para esse individuo: 1.5848931924611134
11º geração: 
Melhor individuo: [-0.04187894  0.9804154 ]
Z para esse individuo: 1.3827860796116198
Fx para esse individuo: 1.5848931924611134
12º geração: 
Melhor individuo: [-0.04187894  0.9804154 ]
Z para esse individuo: 1.3827860796116198
Fx para esse individuo: 1.5848931924611134
13º geração: 
Melhor individuo: [-0.04187894  0.9804154 ]
Z para esse individuo: 1.3827860796116198
Fx para esse individuo: 1.5848931924611134
14º geração: 
Melhor individuo: [-0.04187894  0.9804154 ]
Z para esse individuo: 1.3827860796116198
Fx para esse individuo: 1.5848931924611134
15º geração: 


Melhor individuo: [1.98513018 1.0187344 ]
Z para esse individuo: 5.0913757739213885
Fx para esse individuo: 1.5848931924611134
23º geração: 
Melhor individuo: [1.98513018 1.0187344 ]
Z para esse individuo: 5.0913757739213885
Fx para esse individuo: 1.5848931924611134
24º geração: 
Melhor individuo: [1.98513018 1.0187344 ]
Z para esse individuo: 5.0913757739213885
Fx para esse individuo: 1.5848931924611134
25º geração: 
Melhor individuo: [1.97520432 1.01364084]
Z para esse individuo: 5.08672304337437
Fx para esse individuo: 1.5848931924611134
26º geração: 
Melhor individuo: [1.97520432 1.01364084]
Z para esse individuo: 5.08672304337437
Fx para esse individuo: 1.5848931924611134
27º geração: 
Melhor individuo: [1.9761969 1.0141502]
Z para esse individuo: 5.08498383643923
Fx para esse individuo: 1.5848931924611134
28º geração: 
Melhor individuo: [1.9761969 1.0141502]
Z para esse individuo: 5.08498383643923
Fx para esse individuo: 1.5848931924611134
29º geração: 
Melhor individuo: [1.9770

<h3>Melhores Indivíduos de cada Execução</h3>

In [109]:
bestsIwithC2

array([[-9.80397106e-01,  2.98900750e+00],
       [-1.01216807e+00,  9.77437385e-01],
       [ 3.67630760e-02,  9.93641228e-01],
       [-9.94436914e-01, -9.95545379e-01],
       [-3.42879646e-02,  9.95102851e-01],
       [ 9.55636505e-01, -2.00851874e+00],
       [ 9.84636074e-01,  1.99509098e+00],
       [-1.02455017e+00,  9.63234278e-01],
       [ 9.94378455e-01,  2.40592542e-02],
       [-9.23171861e-01,  1.05707123e+00],
       [ 9.92642120e-01,  2.11876109e-02],
       [ 2.56331077e-03, -9.94953214e-01],
       [-2.48579083e-02, -9.95436347e-01],
       [-9.93419294e-01, -3.94202467e-02],
       [-1.98007737e+00, -1.00548621e+00],
       [-9.93487438e-01,  8.61174263e-03],
       [-4.60143616e-02,  9.91599101e-01],
       [ 4.15236773e-02,  4.02128766e-02],
       [ 1.22296574e-02, -1.98991965e+00],
       [-9.83300900e-01,  1.99556596e+00],
       [-2.99769413e+00, -9.54167499e-01],
       [-9.89495636e-01,  7.55899656e-02],
       [-1.00255601e+00,  4.23771680e-02],
       [ 9.

<h3>Melhores Fitness de cada Execução</h3>

In [110]:
bestsFxwithC2

array([[1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319]])

<h2> Teste: cruzamento3 </h2>

<h3>Melhores de cada geração</h3>

In [111]:
bestsIwithC3, bestsFxwithC3 = algoritimoGenetico(3)


----------1º Execução----------

1º geração: 
Melhor individuo: [ 0.23443753 -1.02092338]
Z para esse individuo: 10.207275624787258
Fx para esse individuo: 1.5848931924611112
2º geração: 
Melhor individuo: [-0.12629509  2.02066867]
Z para esse individuo: 7.169963501989514
Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [0.04564772 1.95290649]
Z para esse individuo: 4.659015455386918
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [0.04564772 1.95290649]
Z para esse individuo: 4.659015455386918
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [0.04857614 1.0472676 ]
Z para esse individuo: 1.9990854875220165
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [0.04857614 1.0472676 ]
Z para esse individuo: 1.9990854875220165
Fx para esse individuo: 1.5848931924611134
7º geração: 
Melhor individuo: [0.04857614 1.0472676 ]
Z para esse individuo: 1.9990854875220165
Fx para esse individuo: 1.58

Fx para esse individuo: 1.5848931924611134
76º geração: 
Melhor individuo: [ 9.94982312e-01 -2.86661042e-04]
Z para esse individuo: 0.9949754710067111
Fx para esse individuo: 1.5848931924611134
77º geração: 
Melhor individuo: [ 9.94982312e-01 -2.86661042e-04]
Z para esse individuo: 0.9949754710067111
Fx para esse individuo: 1.5848931924611134
78º geração: 
Melhor individuo: [ 9.94982312e-01 -2.86661042e-04]
Z para esse individuo: 0.9949754710067111
Fx para esse individuo: 1.5848931924611134
79º geração: 
Melhor individuo: [ 9.94982312e-01 -2.86661042e-04]
Z para esse individuo: 0.9949754710067111
Fx para esse individuo: 1.5848931924611134
80º geração: 
Melhor individuo: [ 9.94982312e-01 -2.86661042e-04]
Z para esse individuo: 0.9949754710067111
Fx para esse individuo: 1.5848931924611134
81º geração: 
Melhor individuo: [ 9.94982312e-01 -2.86661042e-04]
Z para esse individuo: 0.9949754710067111
Fx para esse individuo: 1.5848931924611134
82º geração: 
Melhor individuo: [ 9.94982312e-01 -2

67º geração: 
Melhor individuo: [1.98990887 0.00647041]
Z para esse individuo: 3.9881359852965943
Fx para esse individuo: 1.5848931924611134
68º geração: 
Melhor individuo: [1.98990887 0.00647041]
Z para esse individuo: 3.9881359852965943
Fx para esse individuo: 1.5848931924611134
69º geração: 
Melhor individuo: [1.98990887 0.00647041]
Z para esse individuo: 3.9881359852965943
Fx para esse individuo: 1.5848931924611134
70º geração: 
Melhor individuo: [1.98990887 0.00647041]
Z para esse individuo: 3.9881359852965943
Fx para esse individuo: 1.5848931924611134
71º geração: 
Melhor individuo: [1.98990887 0.00647041]
Z para esse individuo: 3.9881359852965943
Fx para esse individuo: 1.5848931924611134
72º geração: 
Melhor individuo: [1.98990887 0.00647041]
Z para esse individuo: 3.9881359852965943
Fx para esse individuo: 1.5848931924611134
73º geração: 
Melhor individuo: [1.98990887 0.00647041]
Z para esse individuo: 3.9881359852965943
Fx para esse individuo: 1.5848931924611134
74º geração: 

Melhor individuo: [-0.003009   -0.01187473]
Z para esse individuo: 0.029758424262730898
Fx para esse individuo: 1.5848931924611134
33º geração: 
Melhor individuo: [-0.003009   -0.01187473]
Z para esse individuo: 0.029758424262730898
Fx para esse individuo: 1.5848931924611134
34º geração: 
Melhor individuo: [-0.003009   -0.01187473]
Z para esse individuo: 0.029758424262730898
Fx para esse individuo: 1.5848931924611134
35º geração: 
Melhor individuo: [-0.00285855 -0.011281  ]
Z para esse individuo: 0.0268581183823855
Fx para esse individuo: 1.5848931924611134
36º geração: 
Melhor individuo: [-0.00285855 -0.011281  ]
Z para esse individuo: 0.0268581183823855
Fx para esse individuo: 1.5848931924611134
37º geração: 
Melhor individuo: [-0.00285855 -0.011281  ]
Z para esse individuo: 0.0268581183823855
Fx para esse individuo: 1.5848931924611134
38º geração: 
Melhor individuo: [-0.00285855 -0.011281  ]
Z para esse individuo: 0.0268581183823855
Fx para esse individuo: 1.5848931924611134
39º ger

46º geração: 
Melhor individuo: [0.99515296 0.0082147 ]
Z para esse individuo: 1.008351357009147
Fx para esse individuo: 1.5848931924611134
47º geração: 
Melhor individuo: [0.99515296 0.0082147 ]
Z para esse individuo: 1.008351357009147
Fx para esse individuo: 1.5848931924611134
48º geração: 
Melhor individuo: [0.99515296 0.0082147 ]
Z para esse individuo: 1.008351357009147
Fx para esse individuo: 1.5848931924611134
49º geração: 
Melhor individuo: [0.99515296 0.0082147 ]
Z para esse individuo: 1.008351357009147
Fx para esse individuo: 1.5848931924611134
50º geração: 
Melhor individuo: [0.99515296 0.0082147 ]
Z para esse individuo: 1.008351357009147
Fx para esse individuo: 1.5848931924611134
51º geração: 
Melhor individuo: [0.99515296 0.0082147 ]
Z para esse individuo: 1.008351357009147
Fx para esse individuo: 1.5848931924611134
52º geração: 
Melhor individuo: [0.99515296 0.0082147 ]
Z para esse individuo: 1.008351357009147
Fx para esse individuo: 1.5848931924611134
53º geração: 
Melhor

...

Melhor individuo: [-0.04906968 -0.99338796]
Z para esse individuo: 1.4693903363040093
Fx para esse individuo: 1.5848931924611134
59º geração: 
Melhor individuo: [-0.04906968 -0.99338796]
Z para esse individuo: 1.4693903363040093
Fx para esse individuo: 1.5848931924611134
60º geração: 
Melhor individuo: [-0.04906968 -0.99338796]
Z para esse individuo: 1.4693903363040093
Fx para esse individuo: 1.5848931924611134
61º geração: 
Melhor individuo: [-0.04906968 -0.99338796]
Z para esse individuo: 1.4693903363040093
Fx para esse individuo: 1.5848931924611134
62º geração: 
Melhor individuo: [-0.04906968 -0.99338796]
Z para esse individuo: 1.4693903363040093
Fx para esse individuo: 1.5848931924611134
63º geração: 
Melhor individuo: [-0.04906968 -0.99338796]
Z para esse individuo: 1.4693903363040093
Fx para esse individuo: 1.5848931924611134
64º geração: 
Melhor individuo: [-0.04906968 -0.99338796]
Z para esse individuo: 1.4693903363040093
Fx para esse individuo: 1.5848931924611134
65º geração: 

Melhor individuo: [-0.03361654  1.98934714]
Z para esse individuo: 4.20326366742831
Fx para esse individuo: 1.5848931924611134
73º geração: 
Melhor individuo: [-0.03361654  1.98934714]
Z para esse individuo: 4.20326366742831
Fx para esse individuo: 1.5848931924611134
74º geração: 
Melhor individuo: [-0.03361654  1.98934714]
Z para esse individuo: 4.20326366742831
Fx para esse individuo: 1.5848931924611134
75º geração: 
Melhor individuo: [-0.03361654  1.98934714]
Z para esse individuo: 4.20326366742831
Fx para esse individuo: 1.5848931924611134
76º geração: 
Melhor individuo: [-0.03361654  1.98934714]
Z para esse individuo: 4.20326366742831
Fx para esse individuo: 1.5848931924611134
77º geração: 
Melhor individuo: [-0.03361654  1.98934714]
Z para esse individuo: 4.20326366742831
Fx para esse individuo: 1.5848931924611134
78º geração: 
Melhor individuo: [-0.03361654  1.98934714]
Z para esse individuo: 4.20326366742831
Fx para esse individuo: 1.5848931924611134
79º geração: 
Melhor indivi

86º geração: 
Melhor individuo: [-0.00127322  0.99367754]
Z para esse individuo: 0.9956060785459826
Fx para esse individuo: 1.5848931924611134
87º geração: 
Melhor individuo: [-0.00127322  0.99367754]
Z para esse individuo: 0.9956060785459826
Fx para esse individuo: 1.5848931924611134
88º geração: 
Melhor individuo: [-0.00127322  0.99367754]
Z para esse individuo: 0.9956060785459826
Fx para esse individuo: 1.5848931924611134
89º geração: 
Melhor individuo: [-0.00127322  0.99367754]
Z para esse individuo: 0.9956060785459826
Fx para esse individuo: 1.5848931924611134
90º geração: 
Melhor individuo: [-0.00127322  0.99367754]
Z para esse individuo: 0.9956060785459826
Fx para esse individuo: 1.5848931924611134
91º geração: 
Melhor individuo: [-0.00127322  0.99367754]
Z para esse individuo: 0.9956060785459826
Fx para esse individuo: 1.5848931924611134
92º geração: 
Melhor individuo: [-0.00127322  0.99367754]
Z para esse individuo: 0.9956060785459826
Fx para esse individuo: 1.5848931924611134

100º geração: 
Melhor individuo: [0.00635239 0.0095425 ]
Z para esse individuo: 0.026064675180375474
Fx para esse individuo: 1.5848931924611134

----------27º Execução----------

1º geração: 
Melhor individuo: [-0.02290621 -0.99790483]
Z para esse individuo: 1.1005971277570836
Fx para esse individuo: 1.5848931924611134
2º geração: 
Melhor individuo: [-0.02290621 -0.99790483]
Z para esse individuo: 1.1005971277570836
Fx para esse individuo: 1.5848931924611134
3º geração: 
Melhor individuo: [-0.02290621 -0.99790483]
Z para esse individuo: 1.1005971277570836
Fx para esse individuo: 1.5848931924611134
4º geração: 
Melhor individuo: [-0.02290621 -0.99790483]
Z para esse individuo: 1.1005971277570836
Fx para esse individuo: 1.5848931924611134
5º geração: 
Melhor individuo: [-0.02290621 -0.99790483]
Z para esse individuo: 1.1005971277570836
Fx para esse individuo: 1.5848931924611134
6º geração: 
Melhor individuo: [-0.01093565 -1.0033241 ]
Z para esse individuo: 1.0325564077607332
Fx para esse

Fx para esse individuo: 1.5848931924611134
14º geração: 
Melhor individuo: [1.97348745 1.02618829]
Z para esse individuo: 5.2212159914238825
Fx para esse individuo: 1.5848931924611134
15º geração: 
Melhor individuo: [1.97348745 1.02618829]
Z para esse individuo: 5.2212159914238825
Fx para esse individuo: 1.5848931924611134
16º geração: 
Melhor individuo: [1.97348745 1.02618829]
Z para esse individuo: 5.2212159914238825
Fx para esse individuo: 1.5848931924611134
17º geração: 
Melhor individuo: [1.97348745 1.02618829]
Z para esse individuo: 5.2212159914238825
Fx para esse individuo: 1.5848931924611134
18º geração: 
Melhor individuo: [1.97348745 1.02618829]
Z para esse individuo: 5.221215991423881
Fx para esse individuo: 1.5848931924611134
19º geração: 
Melhor individuo: [1.97348745 1.02618829]
Z para esse individuo: 5.221215991423881
Fx para esse individuo: 1.5848931924611134
20º geração: 
Melhor individuo: [1.97348745 1.02618829]
Z para esse individuo: 5.221215991423881
Fx para esse ind

<h3>Melhores Indivíduos de cada Execução</h3>

In [112]:
bestsIwithC3

array([[ 4.60504121e-02,  9.92814620e-01],
       [ 9.94982312e-01, -2.86661042e-04],
       [-1.00339739e+00,  9.86350291e-01],
       [ 1.98990887e+00,  6.47041263e-03],
       [-9.03161542e-03,  1.26017353e-02],
       [-9.65517655e-04, -3.81032464e-03],
       [-5.03573066e-03, -1.94976205e-02],
       [ 9.95087023e-01,  8.21415995e-03],
       [-2.98450215e+00,  3.32934355e-02],
       [ 9.95123893e-01,  9.94869045e-01],
       [ 9.93951945e-01, -1.28084466e-03],
       [ 1.01650415e+00, -9.72278136e-01],
       [ 1.03961182e+00,  9.43606027e-01],
       [-8.73712574e-01, -1.08635187e+00],
       [ 1.00745423e+00,  1.98262241e+00],
       [ 1.71057929e-02,  1.75766550e-02],
       [-1.53440576e-02,  9.94722190e-01],
       [-1.83728443e-03, -3.45287710e-03],
       [ 9.89439729e-01,  2.58795213e-02],
       [-4.90696844e-02, -9.93387964e-01],
       [ 2.01513679e+00, -9.50098878e-01],
       [-3.36165433e-02,  1.98934714e+00],
       [-9.93466659e-01, -3.86764750e-02],
       [-1.

<h3>Melhores Indivíduos de cada Execução</h3>

In [113]:
bestsFxwithC3

array([[1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319],
       [1.58489319, 1.58489319]])

<h1>Comparação dos resultados</h1>

<p>Para verificarmos qual das funções de cruzamento obtive o melhor desempenho, iremos comparar a contagem de indivíduos que convergiram dentre as 30 execuções. Utilizamos como critério de convergência ambas as características do individuo atingirem valores menores que 0.1</p>

In [114]:
cont1 = 0
max1 = 0
cont2 = 0
max2 = 0
cont3 = 0
max3 = 0

for i in range(30):
    if convergeTest(bestsIwithC1[i,0], bestsIwithC1[i,1], 1.5):
        cont1 = cont1 + 1
    if(abs(bestsIwithC1[i,0]) > abs(bestsIwithC1[max1,0]) and abs(bestsIwithC1[i,1]) > abs(bestsIwithC1[max1,1])):
        max1 = i
        
    if convergeTest(bestsIwithC2[i,0], bestsIwithC3[i,1], 1.5):
        cont2 = cont2 + 1
    if(abs(bestsIwithC2[i,0]) > abs(bestsIwithC2[max2,0]) and abs(bestsIwithC2[i,1]) > abs(bestsIwithC2[max2,1])):
        max2 = i
        
    if convergeTest(bestsIwithC3[i,0], bestsIwithC3[i,1], 1.5):
        cont3 = cont3 + 1
    if(abs(bestsIwithC3[i,0]) > abs(bestsIwithC3[max3,0]) and abs(bestsIwithC3[i,1]) > abs(bestsIwithC3[max3,1])):
        max3 = i
        
print("Quantidade de indivíduos que convergiram com o o cruzamento 1: %d" %(cont1))
print("Indivíduo de pior convergência: ")
print(bestsIwithC1[max1])

print("Quantidade de indivíduos que convergiram com o o cruzamento 2: %d" %(cont2))
print("Indivíduo de pior convergência: ")
print(bestsIwithC2[max2])

print("Quantidade de indivíduos que convergiram com o o cruzamento 3: %d" %(cont3))
print("Indivíduo de pior convergência: ")
print(bestsIwithC3[max3])

Quantidade de indivíduos que convergiram com o o cruzamento 1: 19
Indivíduo de pior convergência: 
[ 1.99108697 -1.00238875]
Quantidade de indivíduos que convergiram com o o cruzamento 2: 21
Indivíduo de pior convergência: 
[-0.98039711  2.9890075 ]
Quantidade de indivíduos que convergiram com o o cruzamento 3: 18
Indivíduo de pior convergência: 
[1.00745423 1.98262241]


<h1>Conclusões</h1>

<p>Para a função de cruzamento 1, a convergência ocorreu em cerca de 63% das 30 execuções. Para a função de cruzamento 2, a convergência foi maior 70%. Já a função de cruzamento 3 convergiu em 60% das 30 execuções. O pior resultado na convergência de um indivíduo for encontrado no cruzamento 3.
Vale ressalta que a baixa convergência é esperada devido a simplicidade do algoritmo.</p>